# <center> International Maritime Piracy </center> 


This is part of my EDA process along what I was thinking. This next field (The What, the How) contains the remainder of the EDA questions I have. The code serves only as example as it is contained it /src folder

#### The What

* __Basic Location & Time Trends__
    * What are the year on year trends? 
    * When during the year are attacks most common?
    * Are there any days of the weeks where attaks don't occur (e.g. Friday in MENA or Sunday in Catholic areas)?
 
* __Via Text__
    * How many kidnappings were there?
    * How many murders or killings were there?
    * What about attackers? How may of there were killed?

* __Inland/Offshore__
    * Is it possible to know, using a mask of land coverage, how far from shore attacks occur?
    * What about ones that are more than a mile or so?
    
* __Statistical Analysis__
    * Would like to 
 
#### The How  
 
* __Mapping the Attacks__
    * Need either something with a good API or use something like Folium  
    * (NavArea or SubReg overlay)
    
* __Other Goals__
    * Containerized Web App (AWS, Azure or GCP -- possibly even Heroku or Linode if free)
    

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('../data/piracy.csv')

### See if pandas will read the csv without creating the second line and see if anything is odd.

In [3]:
data.head(5)

,reference,date,position,navArea,subreg,hostility,victim,description,Unnamed: 8
0,2019-73,2019-09-30,"1°02'42""N \n103°39'00""E",XI,71,Five Armed robbers,Bulk Carrier,", SINGAPORE STRAITS. DECK CREW ON ROUTINE ROUN...",NaN
1,2019-72,2019-09-28,"5°19'30""N \n119°43'42""E",XI,92,Two High-speed boats,Bulk Carrier,"AROUND 12NM NW OF TAWI-TAWI ISLAND, PHILIPPINE...",NaN
2,2019-75,2019-09-23,"6°16'60""N \n3°13'60""E",II,57,NaN,NaN,On September 23rd a duty watchman on board a t...,NaN
3,2019-74,2019-09-23,"5°27'53.15""N \n119°07'17.75""E",XI,72,Seven people in Pump Boats,two fishing vessels,TWO FISHING BOATS SAILED TOWARDS TAMBISAN FOR ...,NaN
4,2019-76,2019-09-16,"9°25'00""N \n13°43'60""W",II,51,Four armed robbers,Tanker,On September 16th four robbers armed with a gu...,NaN


### Unnamed: 8 is odd. Is it all NaNs or something else? 

In [4]:
data['Unnamed: 8'].unique()

array([nan])

In [5]:
data.drop(['Unnamed: 8'], axis=1, inplace=True)

In [6]:
data.tail(2)

,reference,date,position,navArea,subreg,hostility,victim,description
7841,1985-2,1978-05-16,"7°59'60""N \n119°59'60""E",XI,92,PIRATES,MERCHANT VESSEL,"SULU SEA: Lloyd's List of 15 May reports a 5,0..."
7842,1989-12,1978-05-01,"5°59'60""N \n119°30'00""E",XI,92,UNKNOWN,UNKNOWN VESSEL,SOUTH SULU SEA ...


### Looks like there are 7825 attacks with all the fields filled out. 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7843 entries, 0 to 7842
Data columns (total 8 columns):
reference      7843 non-null object
date           7843 non-null object
position       7843 non-null object
navArea        7840 non-null object
subreg         7843 non-null int64
hostility      7833 non-null object
victim         7829 non-null object
description    7843 non-null object
dtypes: int64(1), object(7)
memory usage: 490.3+ KB


### Split postion into Lat/Lon in prep for conversion to decimal degrees

In [8]:
# data['position'].str.partition('\n')[[2, 2]].rename({2: 'lat', 3: 'loing'}, axis=1)
# data['Latitude'], data['Longitude'] = data['position'].str.split("\n", expand=True)
# data.join(data['position'].str.split('\n', 1, expand=True).rename(columns={0:'Lat', 1:'Long'}))

# above is part of the graveyard of ineffective pandas code

data = data.join(data['position'].str.split(expand=True)).rename(columns={0:'Latitude', 1:'Longitude'})
data.drop('position', axis=1, inplace=True)


In [9]:
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [10]:
data['Latitude'] = data['Latitude'].apply(dms2dd).round(6)
data['Longitude'] = data['Longitude'].apply(dms2dd).round(6)

In [11]:
data.head(2)

,reference,date,navArea,subreg,hostility,victim,description,Latitude,Longitude
0,2019-73,2019-09-30,XI,71,Five Armed robbers,Bulk Carrier,", SINGAPORE STRAITS. DECK CREW ON ROUTINE ROUN...",1.045,103.650000
1,2019-72,2019-09-28,XI,92,Two High-speed boats,Bulk Carrier,"AROUND 12NM NW OF TAWI-TAWI ISLAND, PHILIPPINE...",5.325,119.728333


### Under 'hostility' - how many different terms are there? 323

In [12]:
data['hostility'].nunique()

327

### Under 'victim' - how many different terms are there? 1224

In [13]:
data['victim'].nunique()


1224

In [14]:
data.to_csv('../data/data_pipe.csv', sep='|', encoding='utf-8')

### Check to see if it worked

In [15]:
ls

EDA for International Maritime Piracy.ipynb
Untitled.ipynb
data.csv
